In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import random as python_random
import json
import argparse
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Embedding, LSTM, Dropout, Bidirectional
from keras.initializers import Constant
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

# Functions

In [ ]:
def read_corpus(corpus_file):
    '''Read in review data set and returns docs and labels'''
    documents = []
    labels = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            tokens = line.strip()
            documents.append(" ".join(tokens.split()[3:]).strip())
            # 6-class problem: books, camera, dvd, health, music, software
            labels.append(tokens.split()[0])
    return documents, labels
    
def read_embeddings(embeddings_file):
    '''Read in word embeddings from file and save as numpy array'''
    embeddings = json.load(open(embeddings_file, 'r'))
    return {word: np.array(embeddings[word]) for word in embeddings}

def get_emb_matrix(voc, emb):
    '''Get embedding matrix given vocab and the embeddings'''
    num_tokens = len(voc) + 2
    word_index = dict(zip(voc, range(len(voc))))
    # Bit hacky, get embedding dimension from the word "the"
    embedding_dim = len(emb["the"])
    # Prepare embedding matrix to the correct size
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = emb.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    # Final matrix with pretrained embeddings that we can feed to embedding layer
    return embedding_matrix

In [ ]:
def train_model(model, X_train, Y_train, X_dev, Y_dev, batch_size, epochs):
    '''Train the model here. Note the different settings you can experiment with!'''
    # Potentially change these to cmd line args again
    # And yes, don't be afraid to experiment!
    verbose = 1
    batch_size = batch_size
    epochs = epochs
    # Early stopping: stop training when there are three consecutive epochs without improving
    # It's also possible to monitor the training loss with monitor="loss"
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    # Finally fit the model to our data
    model.fit(X_train, Y_train, verbose=verbose, epochs=epochs, callbacks=[callback], batch_size=batch_size, validation_data=(X_dev, Y_dev))
    # Print final accuracy for the model (clearer overview)
    test_set_predict(model, X_dev, Y_dev, "dev")
    return model

In [ ]:
def test_set_predict(model, X_test, Y_test, ident):
    '''Do predictions and measure accuracy on our own test set (that we split off train)'''
    # Get predictions using the trained model
    Y_pred = model.predict(X_test)
    # Finally, convert to numerical labels to get scores with sklearn
    Y_pred = np.argmax(Y_pred, axis=1)
    # If you have gold data, you can calculate accuracy
    Y_test = np.argmax(Y_test, axis=1)
    print('Accuracy on own {1} set: {0}'.format(round(accuracy_score(Y_test, Y_pred), 3), ident))

# Data loading

In [ ]:
# data files paths
train_set = '/content/gdrive/MyDrive/LFD-AS3/train.txt'
test_set = '/content/gdrive/MyDrive/LFD-AS3/test.txt'
val_set = '/content/gdrive/MyDrive/LFD-AS3/val.txt'
emb = '/content/gdrive/MyDrive/LFD-AS3/glove_reviews.json'

In [ ]:
# reading training data
X_train, Y_train = read_corpus(train_set)
# reading validation data
X_dev, Y_dev = read_corpus(val_set)
# reading embeddings
embeddings = read_embeddings(emb)

# Text vectorizing + embedding loading + labels encoding

In [ ]:
# Transform words to indices using a vectorizer
vectorizer = TextVectorization(standardize=None, output_sequence_length=50)
# Use train and dev to create vocab - could also do just train
text_ds = tf.data.Dataset.from_tensor_slices(X_train + X_dev)
vectorizer.adapt(text_ds)
# Dictionary mapping words to idx
voc = vectorizer.get_vocabulary()
emb_matrix = get_emb_matrix(voc, embeddings)

# Transform string labels to one-hot encodings
encoder = LabelBinarizer()
Y_train_bin = encoder.fit_transform(Y_train)  # Use encoder.classes_ to find mapping back
Y_dev_bin = encoder.fit_transform(Y_dev)

# Transform input to vectorized input
X_train_vect = vectorizer(np.array([[s] for s in X_train])).numpy()
X_dev_vect = vectorizer(np.array([[s] for s in X_dev])).numpy()

# Read in test set and vectorize
X_test, Y_test = read_corpus(test_set)
Y_test_bin = encoder.fit_transform(Y_test)
X_test_vect = vectorizer(np.array([[s] for s in X_test])).numpy()

# Base model: training + testing

## Experiment 1: One LSTM layer

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 48s 176ms/step - loss: 1.7449 - accuracy: 0.2818 - val_loss: 1.6965 - val_accuracy: 0.4045
Epoch 2/50
264/264 [==============================] - 45s 170ms/step - loss: 1.5271 - accuracy: 0.4609 - val_loss: 1.2433 - val_accuracy: 0.5239
Epoch 3/50
264/264 [==============================] - 45s 171ms/step - loss: 1.1493 - accuracy: 0.5626 - val_loss: 0.9301 - val_accuracy: 0.6477
Epoch 4/50
264/264 [==============================] - 45s 171ms/step - loss: 0.8938 - accuracy: 0.6588 - val_loss: 0.8456 - val_accuracy: 0.6864
Epoch 5/50
264/264 [==============================] - 45s 170ms/step - loss: 0.7456 - accuracy: 0.7256 - val_loss: 0.9572 - val_accuracy: 0.6341
Epoch 6/50
264/264 [==============================] - 45s 171ms/step - loss: 0.6515 - accuracy: 0.7618 - val_loss: 0.6036 - val_accuracy: 0.7795
Epoch 7/50
264/264 [==============================] - 46s 173ms/step - loss: 0.5648 - accuracy: 0.8137 - val_loss: 0.6450 - val_ac

## Experiment 2: One LSTM layer + trainable embedding layer

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=True))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 74s 274ms/step - loss: 1.7447 - accuracy: 0.2813 - val_loss: 1.6956 - val_accuracy: 0.3977
Epoch 2/50
264/264 [==============================] - 64s 242ms/step - loss: 1.5255 - accuracy: 0.4611 - val_loss: 1.2417 - val_accuracy: 0.5227
Epoch 3/50
264/264 [==============================] - 66s 248ms/step - loss: 1.1486 - accuracy: 0.5649 - val_loss: 0.9241 - val_accuracy: 0.6557
Epoch 4/50
264/264 [==============================] - 63s 240ms/step - loss: 0.8934 - accuracy: 0.6609 - val_loss: 0.9020 - val_accuracy: 0.6602
Epoch 5/50
264/264 [==============================] - 63s 240ms/step - loss: 0.7472 - accuracy: 0.7220 - val_loss: 0.8802 - val_accuracy: 0.6557
Epoch 6/50
264/264 [==============================] - 64s 243ms/step - loss: 0.6496 - accuracy: 0.7666 - val_loss: 0.5615 - val_accuracy: 0.8068
Epoch 7/50
264/264 [==============================] - 66s 248ms/step - loss: 0.5624 - accuracy: 0.8055 - val_loss: 0.4937 - val_ac

## Experiment 3: One LSTM layer + one Dense layer between the embedding and the LSTM layers

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 73s 269ms/step - loss: 1.7367 - accuracy: 0.2787 - val_loss: 1.6720 - val_accuracy: 0.3841
Epoch 2/50
264/264 [==============================] - 70s 265ms/step - loss: 1.4396 - accuracy: 0.4493 - val_loss: 1.2202 - val_accuracy: 0.4977
Epoch 3/50
264/264 [==============================] - 68s 259ms/step - loss: 1.1497 - accuracy: 0.5555 - val_loss: 1.0211 - val_accuracy: 0.6398
Epoch 4/50
264/264 [==============================] - 68s 258ms/step - loss: 0.9200 - accuracy: 0.6422 - val_loss: 0.7346 - val_accuracy: 0.7614
Epoch 5/50
264/264 [==============================] - 67s 256ms/step - loss: 0.7107 - accuracy: 0.7443 - val_loss: 0.6868 - val_accuracy: 0.7420
Epoch 6/50
264/264 [==============================] - 67s 255ms/step - loss: 0.6218 - accuracy: 0.7756 - val_loss: 0.6174 - val_accuracy: 0.7875
Epoch 7/50
264/264 [==============================] - 67s 253ms/step - loss: 0.5508 - accuracy: 0.8111 - val_loss: 0.7337 - val_ac

### tanh activation function

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="tanh"))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 66s 244ms/step - loss: 1.7473 - accuracy: 0.2661 - val_loss: 1.6964 - val_accuracy: 0.3614
Epoch 2/50
264/264 [==============================] - 65s 248ms/step - loss: 1.4966 - accuracy: 0.4254 - val_loss: 1.7159 - val_accuracy: 0.3148
Epoch 3/50
264/264 [==============================] - 67s 252ms/step - loss: 1.2184 - accuracy: 0.5232 - val_loss: 1.0447 - val_accuracy: 0.5909
Epoch 4/50
264/264 [==============================] - 68s 256ms/step - loss: 0.9945 - accuracy: 0.6043 - val_loss: 0.8152 - val_accuracy: 0.7307
Epoch 5/50
264/264 [==============================] - 66s 249ms/step - loss: 0.7932 - accuracy: 0.7059 - val_loss: 0.7640 - val_accuracy: 0.6886
Epoch 6/50
264/264 [==============================] - 66s 249ms/step - loss: 0.6759 - accuracy: 0.7566 - val_loss: 0.5704 - val_accuracy: 0.7955
Epoch 7/50
264/264 [==============================] - 64s 243ms/step - loss: 0.6004 - accuracy: 0.7912 - val_loss: 0.6227 - val_ac

### relu activation function

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 63s 231ms/step - loss: 1.7730 - accuracy: 0.2156 - val_loss: 1.7499 - val_accuracy: 0.3057
Epoch 2/50
264/264 [==============================] - 60s 228ms/step - loss: 1.7037 - accuracy: 0.3453 - val_loss: 1.6079 - val_accuracy: 0.3989
Epoch 3/50
264/264 [==============================] - 61s 229ms/step - loss: 1.4107 - accuracy: 0.4050 - val_loss: 1.2690 - val_accuracy: 0.5114
Epoch 4/50
264/264 [==============================] - 61s 230ms/step - loss: 1.2469 - accuracy: 0.4874 - val_loss: 1.1541 - val_accuracy: 0.4784
Epoch 5/50
264/264 [==============================] - 61s 230ms/step - loss: 1.0362 - accuracy: 0.5900 - val_loss: 0.8441 - val_accuracy: 0.7057
Epoch 6/50
264/264 [==============================] - 60s 229ms/step - loss: 0.8197 - accuracy: 0.6699 - val_loss: 0.8363 - val_accuracy: 0.6375
Epoch 7/50
264/264 [==============================] - 61s 231ms/step - loss: 0.6346 - accuracy: 0.7704 - val_loss: 0.6260 - val_ac

sigmoid activation function

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="sigmoid"))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 65s 241ms/step - loss: 1.8067 - accuracy: 0.1784 - val_loss: 1.7928 - val_accuracy: 0.1648
Epoch 2/50
264/264 [==============================] - 61s 233ms/step - loss: 1.8009 - accuracy: 0.1827 - val_loss: 1.8071 - val_accuracy: 0.2091
Epoch 3/50
264/264 [==============================] - 60s 226ms/step - loss: 1.7943 - accuracy: 0.1848 - val_loss: 1.7930 - val_accuracy: 0.1636
Epoch 4/50
264/264 [==============================] - 60s 227ms/step - loss: 1.7876 - accuracy: 0.1941 - val_loss: 1.7907 - val_accuracy: 0.1716
Epoch 5/50
264/264 [==============================] - 60s 226ms/step - loss: 1.7819 - accuracy: 0.1953 - val_loss: 1.7841 - val_accuracy: 0.1875
Epoch 6/50
264/264 [==============================] - 60s 226ms/step - loss: 1.7774 - accuracy: 0.2000 - val_loss: 1.7658 - val_accuracy: 0.2000
Epoch 7/50
264/264 [==============================] - 60s 226ms/step - loss: 1.7670 - accuracy: 0.2182 - val_loss: 1.7733 - val_ac

softmax activation function

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="softmax"))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 65s 238ms/step - loss: 1.7923 - accuracy: 0.1590 - val_loss: 1.7917 - val_accuracy: 0.1716
Epoch 2/50
264/264 [==============================] - 62s 235ms/step - loss: 1.7923 - accuracy: 0.1687 - val_loss: 1.7917 - val_accuracy: 0.1682
Epoch 3/50
264/264 [==============================] - 62s 236ms/step - loss: 1.7923 - accuracy: 0.1664 - val_loss: 1.7917 - val_accuracy: 0.1682
Epoch 4/50
264/264 [==============================] - 63s 239ms/step - loss: 1.7923 - accuracy: 0.1649 - val_loss: 1.7918 - val_accuracy: 0.1716
Accuracy on own dev set: 0.172
Accuracy on own test set: 0.171


## Experiment 4: Extra LSTM layers

### One extra LSTM layer

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, return_sequences=True))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 129s 476ms/step - loss: 1.7773 - accuracy: 0.2130 - val_loss: 1.7573 - val_accuracy: 0.3341
Epoch 2/50
264/264 [==============================] - 118s 448ms/step - loss: 1.6835 - accuracy: 0.3583 - val_loss: 1.4323 - val_accuracy: 0.4216
Epoch 3/50
264/264 [==============================] - 117s 445ms/step - loss: 1.3912 - accuracy: 0.3983 - val_loss: 1.2605 - val_accuracy: 0.4580
Epoch 4/50
264/264 [==============================] - 117s 444ms/step - loss: 1.2380 - accuracy: 0.4756 - val_loss: 1.1447 - val_accuracy: 0.4795
Epoch 5/50
264/264 [==============================] - 118s 448ms/step - loss: 0.9846 - accuracy: 0.6076 - val_loss: 0.9101 - val_accuracy: 0.6057
Epoch 6/50
264/264 [==============================] - 118s 446ms/step - loss: 0.7356 - accuracy: 0.7159 - val_loss: 0.6141 - val_accuracy: 0.7909
Epoch 7/50
264/264 [==============================] - 117s 442ms/step - loss: 0.5717 - accuracy: 0.7924 - val_loss: 0.5943 -

### Two extra LSTM layers

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, return_sequences=True))
    model.add(LSTM(embedding_dim, return_sequences=True))
    model.add(LSTM(embedding_dim))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 177s 653ms/step - loss: 1.7829 - accuracy: 0.1934 - val_loss: 1.7681 - val_accuracy: 0.2886
Epoch 2/50
264/264 [==============================] - 171s 649ms/step - loss: 1.7129 - accuracy: 0.3424 - val_loss: 1.4704 - val_accuracy: 0.3761
Epoch 3/50
264/264 [==============================] - 171s 649ms/step - loss: 1.4181 - accuracy: 0.3787 - val_loss: 1.2568 - val_accuracy: 0.4693
Epoch 4/50
264/264 [==============================] - 172s 651ms/step - loss: 1.1986 - accuracy: 0.4813 - val_loss: 0.9516 - val_accuracy: 0.6341
Epoch 5/50
264/264 [==============================] - 172s 651ms/step - loss: 0.9281 - accuracy: 0.6291 - val_loss: 0.8137 - val_accuracy: 0.7034
Epoch 6/50
264/264 [==============================] - 172s 650ms/step - loss: 0.7035 - accuracy: 0.7419 - val_loss: 0.7151 - val_accuracy: 0.7068
Epoch 7/50
264/264 [==============================] - 172s 650ms/step - loss: 0.5860 - accuracy: 0.7886 - val_loss: 0.6009 -

## Experiment 5: Dropout and Recurrent Dropout

### dropout = 0.1

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.1))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 86s 317ms/step - loss: 1.7738 - accuracy: 0.2166 - val_loss: 1.7504 - val_accuracy: 0.3068
Epoch 2/50
264/264 [==============================] - 82s 312ms/step - loss: 1.7076 - accuracy: 0.3408 - val_loss: 1.6245 - val_accuracy: 0.4000
Epoch 3/50
264/264 [==============================] - 81s 307ms/step - loss: 1.4135 - accuracy: 0.4031 - val_loss: 1.2697 - val_accuracy: 0.5193
Epoch 4/50
264/264 [==============================] - 80s 304ms/step - loss: 1.2505 - accuracy: 0.4813 - val_loss: 1.1546 - val_accuracy: 0.4580
Epoch 5/50
264/264 [==============================] - 81s 306ms/step - loss: 1.0493 - accuracy: 0.5794 - val_loss: 0.8757 - val_accuracy: 0.6955
Epoch 6/50
264/264 [==============================] - 82s 309ms/step - loss: 0.8371 - accuracy: 0.6675 - val_loss: 0.7763 - val_accuracy: 0.6750
Epoch 7/50
264/264 [==============================] - 81s 308ms/step - loss: 0.6505 - accuracy: 0.7602 - val_loss: 0.6367 - val_ac

### dropout = 0.2

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.2))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 67s 248ms/step - loss: 1.7749 - accuracy: 0.2076 - val_loss: 1.7511 - val_accuracy: 0.3091
Epoch 2/50
264/264 [==============================] - 68s 256ms/step - loss: 1.7123 - accuracy: 0.3348 - val_loss: 1.6413 - val_accuracy: 0.3886
Epoch 3/50
264/264 [==============================] - 67s 252ms/step - loss: 1.4342 - accuracy: 0.4007 - val_loss: 1.3112 - val_accuracy: 0.5102
Epoch 4/50
264/264 [==============================] - 66s 250ms/step - loss: 1.2527 - accuracy: 0.4773 - val_loss: 1.1639 - val_accuracy: 0.4432
Epoch 5/50
264/264 [==============================] - 67s 253ms/step - loss: 1.0691 - accuracy: 0.5678 - val_loss: 0.8934 - val_accuracy: 0.6807
Epoch 6/50
264/264 [==============================] - 67s 252ms/step - loss: 0.8475 - accuracy: 0.6590 - val_loss: 0.8338 - val_accuracy: 0.6432
Epoch 7/50
264/264 [==============================] - 66s 250ms/step - loss: 0.6627 - accuracy: 0.7500 - val_loss: 0.7415 - val_ac

### dropout = 0.3

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.3))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 70s 257ms/step - loss: 1.7759 - accuracy: 0.2097 - val_loss: 1.7517 - val_accuracy: 0.3057
Epoch 2/50
264/264 [==============================] - 67s 253ms/step - loss: 1.7165 - accuracy: 0.3294 - val_loss: 1.6546 - val_accuracy: 0.3807
Epoch 3/50
264/264 [==============================] - 66s 251ms/step - loss: 1.4556 - accuracy: 0.3979 - val_loss: 1.2920 - val_accuracy: 0.5227
Epoch 4/50
264/264 [==============================] - 67s 253ms/step - loss: 1.2630 - accuracy: 0.4739 - val_loss: 1.1682 - val_accuracy: 0.4364
Epoch 5/50
264/264 [==============================] - 66s 252ms/step - loss: 1.0862 - accuracy: 0.5618 - val_loss: 0.9164 - val_accuracy: 0.6602
Epoch 6/50
264/264 [==============================] - 67s 254ms/step - loss: 0.8610 - accuracy: 0.6547 - val_loss: 0.8288 - val_accuracy: 0.6511
Epoch 7/50
264/264 [==============================] - 67s 254ms/step - loss: 0.6806 - accuracy: 0.7448 - val_loss: 0.7341 - val_ac

### dropout = 0.4

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.4))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 66s 240ms/step - loss: 1.7771 - accuracy: 0.2078 - val_loss: 1.7523 - val_accuracy: 0.3068
Epoch 2/50
264/264 [==============================] - 64s 242ms/step - loss: 1.7203 - accuracy: 0.3218 - val_loss: 1.6660 - val_accuracy: 0.3761
Epoch 3/50
264/264 [==============================] - 64s 242ms/step - loss: 1.4709 - accuracy: 0.4036 - val_loss: 1.2804 - val_accuracy: 0.5148
Epoch 4/50
264/264 [==============================] - 63s 240ms/step - loss: 1.2680 - accuracy: 0.4713 - val_loss: 1.1716 - val_accuracy: 0.4261
Epoch 5/50
264/264 [==============================] - 63s 239ms/step - loss: 1.1025 - accuracy: 0.5545 - val_loss: 0.9883 - val_accuracy: 0.6216
Epoch 6/50
264/264 [==============================] - 65s 248ms/step - loss: 0.8896 - accuracy: 0.6448 - val_loss: 0.9564 - val_accuracy: 0.5807
Epoch 7/50
264/264 [==============================] - 65s 245ms/step - loss: 0.6990 - accuracy: 0.7358 - val_loss: 0.7200 - val_ac

### dropout = 0.35

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 64s 234ms/step - loss: 1.7765 - accuracy: 0.2076 - val_loss: 1.7520 - val_accuracy: 0.3080
Epoch 2/50
264/264 [==============================] - 66s 249ms/step - loss: 1.7180 - accuracy: 0.3265 - val_loss: 1.6597 - val_accuracy: 0.3784
Epoch 3/50
264/264 [==============================] - 66s 248ms/step - loss: 1.4579 - accuracy: 0.3962 - val_loss: 1.2929 - val_accuracy: 0.5216
Epoch 4/50
264/264 [==============================] - 66s 251ms/step - loss: 1.2836 - accuracy: 0.4730 - val_loss: 1.1732 - val_accuracy: 0.4284
Epoch 5/50
264/264 [==============================] - 65s 247ms/step - loss: 1.0949 - accuracy: 0.5569 - val_loss: 0.9378 - val_accuracy: 0.6580
Epoch 6/50
264/264 [==============================] - 66s 251ms/step - loss: 0.8740 - accuracy: 0.6536 - val_loss: 1.0154 - val_accuracy: 0.5614
Epoch 7/50
264/264 [==============================] - 66s 252ms/step - loss: 0.6926 - accuracy: 0.7429 - val_loss: 0.6967 - val_ac

### recurrent dropout = 0.1

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.1))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 142s 528ms/step - loss: 1.7729 - accuracy: 0.2164 - val_loss: 1.7499 - val_accuracy: 0.3091
Epoch 2/50
264/264 [==============================] - 138s 522ms/step - loss: 1.7043 - accuracy: 0.3429 - val_loss: 1.6108 - val_accuracy: 0.4011
Epoch 3/50
264/264 [==============================] - 138s 522ms/step - loss: 1.4134 - accuracy: 0.4100 - val_loss: 1.2852 - val_accuracy: 0.5091
Epoch 4/50
264/264 [==============================] - 138s 524ms/step - loss: 1.2413 - accuracy: 0.4879 - val_loss: 1.1577 - val_accuracy: 0.4670
Epoch 5/50
264/264 [==============================] - 139s 525ms/step - loss: 1.0396 - accuracy: 0.5900 - val_loss: 0.8846 - val_accuracy: 0.6875
Epoch 6/50
264/264 [==============================] - 137s 519ms/step - loss: 0.8261 - accuracy: 0.6704 - val_loss: 0.7230 - val_accuracy: 0.7068
Epoch 7/50
264/264 [==============================] - 137s 520ms/step - loss: 0.6415 - accuracy: 0.7618 - val_loss: 0.7044 -

### recurrent dropout = 0.2

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.2))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 153s 572ms/step - loss: 1.7729 - accuracy: 0.2164 - val_loss: 1.7500 - val_accuracy: 0.3080
Epoch 2/50
264/264 [==============================] - 140s 529ms/step - loss: 1.7046 - accuracy: 0.3445 - val_loss: 1.6116 - val_accuracy: 0.3989
Epoch 3/50
264/264 [==============================] - 136s 517ms/step - loss: 1.4098 - accuracy: 0.4062 - val_loss: 1.2633 - val_accuracy: 0.5102
Epoch 4/50
264/264 [==============================] - 139s 526ms/step - loss: 1.2489 - accuracy: 0.4872 - val_loss: 1.1559 - val_accuracy: 0.4705
Epoch 5/50
264/264 [==============================] - 138s 523ms/step - loss: 1.0471 - accuracy: 0.5829 - val_loss: 0.8930 - val_accuracy: 0.6739
Epoch 6/50
264/264 [==============================] - 138s 521ms/step - loss: 0.8292 - accuracy: 0.6732 - val_loss: 0.8896 - val_accuracy: 0.6091
Epoch 7/50
264/264 [==============================] - 138s 523ms/step - loss: 0.6402 - accuracy: 0.7659 - val_loss: 0.6949 -

### recurrent dropout = 0.3

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.3))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 144s 537ms/step - loss: 1.7729 - accuracy: 0.2164 - val_loss: 1.7500 - val_accuracy: 0.3068
Epoch 2/50
264/264 [==============================] - 138s 521ms/step - loss: 1.7049 - accuracy: 0.3457 - val_loss: 1.6144 - val_accuracy: 0.3966
Epoch 3/50
264/264 [==============================] - 145s 550ms/step - loss: 1.4074 - accuracy: 0.4059 - val_loss: 1.2801 - val_accuracy: 0.5125
Epoch 4/50
264/264 [==============================] - 142s 536ms/step - loss: 1.2383 - accuracy: 0.4820 - val_loss: 1.1574 - val_accuracy: 0.4614
Epoch 5/50
264/264 [==============================] - 144s 547ms/step - loss: 1.0492 - accuracy: 0.5879 - val_loss: 0.8794 - val_accuracy: 0.6807
Epoch 6/50
264/264 [==============================] - 141s 533ms/step - loss: 0.8379 - accuracy: 0.6687 - val_loss: 0.6943 - val_accuracy: 0.7216
Epoch 7/50
264/264 [==============================] - 148s 560ms/step - loss: 0.6483 - accuracy: 0.7602 - val_loss: 0.6817 -

### recurrent dropout = 0.35

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 146s 544ms/step - loss: 1.7731 - accuracy: 0.2159 - val_loss: 1.7500 - val_accuracy: 0.3091
Epoch 2/50
264/264 [==============================] - 145s 550ms/step - loss: 1.7050 - accuracy: 0.3429 - val_loss: 1.6162 - val_accuracy: 0.4000
Epoch 3/50
264/264 [==============================] - 144s 547ms/step - loss: 1.4101 - accuracy: 0.4038 - val_loss: 1.2682 - val_accuracy: 0.5182
Epoch 4/50
264/264 [==============================] - 145s 550ms/step - loss: 1.2413 - accuracy: 0.4841 - val_loss: 1.1587 - val_accuracy: 0.4670
Epoch 5/50
264/264 [==============================] - 141s 536ms/step - loss: 1.0550 - accuracy: 0.5808 - val_loss: 0.9163 - val_accuracy: 0.6511
Epoch 6/50
264/264 [==============================] - 143s 542ms/step - loss: 0.8375 - accuracy: 0.6661 - val_loss: 0.7281 - val_accuracy: 0.7034
Epoch 7/50
264/264 [==============================] - 144s 546ms/step - loss: 0.6590 - accuracy: 0.7581 - val_loss: 0.6889 -

### recurrent dropout = 0.4

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.4))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 143s 535ms/step - loss: 1.7731 - accuracy: 0.2197 - val_loss: 1.7500 - val_accuracy: 0.3080
Epoch 2/50
264/264 [==============================] - 142s 538ms/step - loss: 1.7057 - accuracy: 0.3398 - val_loss: 1.6199 - val_accuracy: 0.3932
Epoch 3/50
264/264 [==============================] - 142s 539ms/step - loss: 1.4103 - accuracy: 0.4069 - val_loss: 1.2731 - val_accuracy: 0.5125
Epoch 4/50
264/264 [==============================] - 140s 531ms/step - loss: 1.2337 - accuracy: 0.4841 - val_loss: 1.1638 - val_accuracy: 0.4602
Epoch 5/50
264/264 [==============================] - 143s 540ms/step - loss: 1.0596 - accuracy: 0.5841 - val_loss: 0.9355 - val_accuracy: 0.6636
Epoch 6/50
264/264 [==============================] - 144s 545ms/step - loss: 0.8386 - accuracy: 0.6652 - val_loss: 0.7674 - val_accuracy: 0.6818
Epoch 7/50
264/264 [==============================] - 143s 543ms/step - loss: 0.6569 - accuracy: 0.7540 - val_loss: 0.6623 -

### recurrent dropout = 0.3 + dropout=0.35

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.3, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 149s 555ms/step - loss: 1.7745 - accuracy: 0.2178 - val_loss: 1.7518 - val_accuracy: 0.3057
Epoch 2/50
264/264 [==============================] - 144s 545ms/step - loss: 1.7181 - accuracy: 0.3206 - val_loss: 1.6589 - val_accuracy: 0.3670
Epoch 3/50
264/264 [==============================] - 147s 557ms/step - loss: 1.4644 - accuracy: 0.4024 - val_loss: 1.2940 - val_accuracy: 0.5182
Epoch 4/50
264/264 [==============================] - 145s 548ms/step - loss: 1.2760 - accuracy: 0.4697 - val_loss: 1.1822 - val_accuracy: 0.4250
Epoch 5/50
264/264 [==============================] - 144s 545ms/step - loss: 1.1025 - accuracy: 0.5763 - val_loss: 0.9837 - val_accuracy: 0.6159
Epoch 6/50
264/264 [==============================] - 147s 556ms/step - loss: 0.8900 - accuracy: 0.6410 - val_loss: 0.7798 - val_accuracy: 0.6807
Epoch 7/50
264/264 [==============================] - 146s 552ms/step - loss: 0.7036 - accuracy: 0.7322 - val_loss: 0.6333 -

### recurrent dropout = 0.1 + dropout=0.35

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = 0.01
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, recurrent_dropout=0.1, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

# Create model
model = create_model(Y_train, emb_matrix)
# Train the model
model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
# Finally do the predictions
test_set_predict(model, X_test_vect, Y_test_bin, "test")

Epoch 1/50
264/264 [==============================] - 152s 559ms/step - loss: 1.7738 - accuracy: 0.2182 - val_loss: 1.7516 - val_accuracy: 0.3080
Epoch 2/50
264/264 [==============================] - 159s 602ms/step - loss: 1.7176 - accuracy: 0.3204 - val_loss: 1.6587 - val_accuracy: 0.3750
Epoch 3/50
264/264 [==============================] - 158s 597ms/step - loss: 1.4558 - accuracy: 0.4045 - val_loss: 1.3070 - val_accuracy: 0.5091
Epoch 4/50
264/264 [==============================] - 156s 590ms/step - loss: 1.2647 - accuracy: 0.4858 - val_loss: 1.1721 - val_accuracy: 0.4409
Epoch 5/50
264/264 [==============================] - 150s 568ms/step - loss: 1.0939 - accuracy: 0.5727 - val_loss: 0.9185 - val_accuracy: 0.6409
Epoch 6/50
264/264 [==============================] - 159s 604ms/step - loss: 0.8732 - accuracy: 0.6464 - val_loss: 0.8504 - val_accuracy: 0.6216
Epoch 7/50
264/264 [==============================] - 162s 613ms/step - loss: 0.6834 - accuracy: 0.7453 - val_loss: 0.7675 -

## Experiment 6: Optimizers - learning rate

### SGD Optimizer

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix, lr):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = lr
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

lrs = [0.01, 0.03, 0.07, 0.1, 0.2, 0.5]
for lr in lrs:
  print(lr)
  print("---")
  # Create model
  model = create_model(Y_train, emb_matrix, lr)
  # Train the model
  model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
  # Finally do the predictions
  test_set_predict(model, X_test_vect, Y_test_bin, "test")
  print("---------------------------")

0.01
---
Epoch 1/50
264/264 [==============================] - 11s 10ms/step - loss: 1.7765 - accuracy: 0.2076 - val_loss: 1.7520 - val_accuracy: 0.3080
Epoch 2/50
264/264 [==============================] - 2s 7ms/step - loss: 1.7180 - accuracy: 0.3265 - val_loss: 1.6597 - val_accuracy: 0.3784
Epoch 3/50
264/264 [==============================] - 3s 11ms/step - loss: 1.4579 - accuracy: 0.3960 - val_loss: 1.2934 - val_accuracy: 0.5216
Epoch 4/50
264/264 [==============================] - 3s 13ms/step - loss: 1.2582 - accuracy: 0.4746 - val_loss: 1.1692 - val_accuracy: 0.4375
Epoch 5/50
264/264 [==============================] - 2s 8ms/step - loss: 1.0899 - accuracy: 0.5585 - val_loss: 0.9460 - val_accuracy: 0.6557
Epoch 6/50
264/264 [==============================] - 2s 7ms/step - loss: 0.8686 - accuracy: 0.6573 - val_loss: 0.8765 - val_accuracy: 0.6307
Epoch 7/50
264/264 [==============================] - 2s 7ms/step - loss: 0.6885 - accuracy: 0.7422 - val_loss: 0.6864 - val_accuracy: 

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix, lr):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = lr
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

lrs = [0.01, 0.003, 0.007, 0.005]
for lr in lrs:
  print(lr)
  print("---")
  # Create model
  model = create_model(Y_train, emb_matrix, lr)
  # Train the model
  model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
  # Finally do the predictions
  test_set_predict(model, X_test_vect, Y_test_bin, "test")
  print("---------------------------")

0.01
---
Epoch 1/50
264/264 [==============================] - 4s 10ms/step - loss: 1.7765 - accuracy: 0.2076 - val_loss: 1.7520 - val_accuracy: 0.3080
Epoch 2/50
264/264 [==============================] - 2s 8ms/step - loss: 1.7180 - accuracy: 0.3265 - val_loss: 1.6597 - val_accuracy: 0.3784
Epoch 3/50
264/264 [==============================] - 2s 7ms/step - loss: 1.4581 - accuracy: 0.3950 - val_loss: 1.2947 - val_accuracy: 0.5239
Epoch 4/50
264/264 [==============================] - 2s 7ms/step - loss: 1.2655 - accuracy: 0.4770 - val_loss: 1.1697 - val_accuracy: 0.4375
Epoch 5/50
264/264 [==============================] - 2s 7ms/step - loss: 1.0898 - accuracy: 0.5564 - val_loss: 0.9475 - val_accuracy: 0.6534
Epoch 6/50
264/264 [==============================] - 2s 7ms/step - loss: 0.8702 - accuracy: 0.6547 - val_loss: 1.0016 - val_accuracy: 0.5625
Epoch 7/50
264/264 [==============================] - 2s 7ms/step - loss: 0.6909 - accuracy: 0.7424 - val_loss: 0.6914 - val_accuracy: 0.7

### Adam Optimizer

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix, lr):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = lr
    loss_function = 'categorical_crossentropy'
    optim = Adam(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

lrs = [0.01, 0.03, 0.07, 0.1, 0.2, 0.5]
for lr in lrs:
  print(lr)
  print("---")
  # Create model
  model = create_model(Y_train, emb_matrix, lr)
  # Train the model
  model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
  # Finally do the predictions
  test_set_predict(model, X_test_vect, Y_test_bin, "test")
  print("---------------------------")

0.01
---
Epoch 1/50
264/264 [==============================] - 5s 10ms/step - loss: 0.8057 - accuracy: 0.7045 - val_loss: 0.4075 - val_accuracy: 0.8773
Epoch 2/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3998 - accuracy: 0.8621 - val_loss: 0.3684 - val_accuracy: 0.8795
Epoch 3/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3517 - accuracy: 0.8813 - val_loss: 0.4056 - val_accuracy: 0.8841
Epoch 4/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3476 - accuracy: 0.8810 - val_loss: 0.4176 - val_accuracy: 0.8682
Epoch 5/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3388 - accuracy: 0.8848 - val_loss: 0.4143 - val_accuracy: 0.8659
Accuracy on own dev set: 0.866
Accuracy on own test set: 0.881
---------------------------
0.03
---
Epoch 1/50
264/264 [==============================] - 4s 10ms/step - loss: 1.8988 - accuracy: 0.2209 - val_loss: 1.6695 - val_accuracy: 0.2739
Epoch 2/50
264/264 [=================

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix, lr):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = lr
    loss_function = 'categorical_crossentropy'
    optim = Adam(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(LSTM(embedding_dim, dropout=0.35))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

lrs = [0.01, 0.003, 0.007, 0.005]
for lr in lrs:
  print(lr)
  print("---")
  # Create model
  model = create_model(Y_train, emb_matrix, lr)
  # Train the model
  model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
  # Finally do the predictions
  test_set_predict(model, X_test_vect, Y_test_bin, "test")
  print("---------------------------")

0.01
---
Epoch 1/50
264/264 [==============================] - 4s 10ms/step - loss: 0.8163 - accuracy: 0.6945 - val_loss: 0.3964 - val_accuracy: 0.8795
Epoch 2/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3844 - accuracy: 0.8735 - val_loss: 0.4384 - val_accuracy: 0.8716
Epoch 3/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3668 - accuracy: 0.8787 - val_loss: 0.3885 - val_accuracy: 0.8795
Epoch 4/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3643 - accuracy: 0.8739 - val_loss: 0.3987 - val_accuracy: 0.8750
Epoch 5/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3535 - accuracy: 0.8822 - val_loss: 0.4481 - val_accuracy: 0.8693
Epoch 6/50
264/264 [==============================] - 2s 8ms/step - loss: 0.3341 - accuracy: 0.8808 - val_loss: 0.4045 - val_accuracy: 0.8761
Accuracy on own dev set: 0.876
Accuracy on own test set: 0.863
---------------------------
0.003
---
Epoch 1/50
264/264 [=================

## Experiment 7: Bi-directional LSTM

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
python_random.seed(1234)

def create_model(Y_train, emb_matrix, lr):
    '''Create the Keras model to use'''
    # Define settings, you might want to create cmd line args for them
    learning_rate = lr
    loss_function = 'categorical_crossentropy'
    optim = SGD(learning_rate=learning_rate)
    # Take embedding dim and size from emb_matrix
    embedding_dim = len(emb_matrix[0])
    num_tokens = len(emb_matrix)
    num_labels = len(set(Y_train))
    # Now build the model
    model = Sequential()
    model.add(Embedding(num_tokens, embedding_dim, embeddings_initializer=Constant(emb_matrix),trainable=False))
    # Here you should add LSTM layers (and potentially dropout)
    model.add(Dense(embedding_dim, activation="relu"))
    model.add(Bidirectional(LSTM(embedding_dim, dropout=0.35)))
    # Ultimately, end with dense layer with softmax
    model.add(Dense(input_dim=embedding_dim, units=num_labels, activation="softmax"))
    # Compile model using our settings, check for accuracy
    model.compile(loss=loss_function, optimizer=optim, metrics=['accuracy'])
    return model

lrs = [0.01, 0.03, 0.07, 0.1, 0.2, 0.5]
for lr in lrs:
  print(lr)
  print("---")
  # Create model
  model = create_model(Y_train, emb_matrix, lr)
  # Train the model
  model = train_model(model, X_train_vect, Y_train_bin, X_dev_vect, Y_dev_bin, 16, 50)
  # Finally do the predictions
  test_set_predict(model, X_test_vect, Y_test_bin, "test")
  print("---------------------------")

0.01
---
Epoch 1/50
264/264 [==============================] - 7s 15ms/step - loss: 1.7575 - accuracy: 0.2595 - val_loss: 1.7125 - val_accuracy: 0.4205
Epoch 2/50
264/264 [==============================] - 3s 12ms/step - loss: 1.6716 - accuracy: 0.4386 - val_loss: 1.5841 - val_accuracy: 0.5205
Epoch 3/50
264/264 [==============================] - 3s 12ms/step - loss: 1.3863 - accuracy: 0.5415 - val_loss: 1.2195 - val_accuracy: 0.6568
Epoch 4/50
264/264 [==============================] - 3s 12ms/step - loss: 1.0862 - accuracy: 0.6092 - val_loss: 0.8452 - val_accuracy: 0.7091
Epoch 5/50
264/264 [==============================] - 3s 12ms/step - loss: 0.8482 - accuracy: 0.6874 - val_loss: 0.6690 - val_accuracy: 0.7625
Epoch 6/50
264/264 [==============================] - 3s 12ms/step - loss: 0.6852 - accuracy: 0.7576 - val_loss: 0.5685 - val_accuracy: 0.7989
Epoch 7/50
264/264 [==============================] - 3s 12ms/step - loss: 0.5568 - accuracy: 0.8066 - val_loss: 0.5277 - val_accurac